<a href="https://colab.research.google.com/github/salmabenhassin/CognoRise-Infotech-Projects/blob/main/Sentiment_Analysis_on_IMDB_Reviews_with_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [ ]:
import os
import json

from zipfile import ZipFile
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding , LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

Data collection : kaggle API

In [ ]:
kaggle_dictionnary = json.load(open('kaggle.json'))
print(kaggle_dictionnary)
kaggle_dictionnary.keys()


{'username': 'salmabenhssin', 'key': 'ba33c8df35219bcd61c9826dcb51d3f2'}


dict_keys(['username', 'key'])

In [ ]:
#setup kaggle credentials(api) as environment variables
#ce sont les variables d'environ: 'KAGGLE_USERNAME,KAGGLE_KEY'
os.environ['KAGGLE_USERNAME'] = kaggle_dictionnary['username']
os.environ['KAGGLE_KEY'] = kaggle_dictionnary['key']

In [ ]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
 97% 25.0M/25.7M [00:02<00:00, 21.5MB/s]
100% 25.7M/25.7M [00:02<00:00, 11.9MB/s]


In [ ]:
!ls

imdb-dataset-of-50k-movie-reviews.zip  kaggle.json  sample_data


In [ ]:
#unzip the dataset file

with ZipFile('imdb-dataset-of-50k-movie-reviews.zip','r') as zip_ref:
  zip_ref.extractall()

In [ ]:
!ls

'IMDB Dataset.csv'   imdb-dataset-of-50k-movie-reviews.zip   kaggle.json   sample_data


loading the dataset


In [ ]:
data = pd.read_csv('IMDB Dataset.csv')

In [ ]:
data.shape

(50000, 2)

In [ ]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [ ]:
data['sentiment'].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [ ]:
#convertit sentiment en val numerique tq 1 postive , en creant un dict contenant autre dict
data.replace({"sentiment" : {'positive':1,'negative':0}},inplace=True)
#inplace : impose de remplacer ca ds dataframe !!



In [ ]:
data.head()


,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [ ]:
#spli data into train data and test data
train_data , test_data = train_test_split(data,test_size=0.2, random_state=42)

In [ ]:
print(train_data.shape)
print(test_data.shape)


(40000, 2)
(10000, 2)


data preprocessing

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

example = data['review'][100]
print(example)
tokens = nltk.word_tokenize(example)
tokens[:10]
tagged = nltk.pos_tag(tokens)
tagged[:10]
nltk.download('maxent_ne_chunker')
nltk.download('words')
entities = nltk.chunk.ne_chunk(tagged)
entities.pprint()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...


This short film that inspired the soon-to-be full length feature - Spatula Madness - is a hilarious piece that contends against similar cartoons yielding multiple writers. The short film stars Edward the Spatula who after being fired from his job, joins in the fight against the evil spoons. This premise allows for some funny content near the beginning, but is barely present for the remainder of the feature. This film's 15-minute running time is absorbed by some odd-ball comedy and a small musical number. Unfortunately not much else lies below it. The plot that is set up doesn't really have time to show. But it's surely follows it plot better than many high-budget Hollywood films. This film is worth watching at least a few times. Take it for what it is, and don't expect a deep story.


[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...


(S
  This/DT
  short/JJ
  film/NN
  that/WDT
  inspired/VBD
  the/DT
  soon-to-be/JJ
  full/JJ
  length/NN
  feature/NN
  -/:
  (PERSON Spatula/NNP Madness/NNP)
  -/:
  is/VBZ
  a/DT
  hilarious/JJ
  piece/NN
  that/WDT
  contends/VBZ
  against/IN
  similar/JJ
  cartoons/NNS
  yielding/VBG
  multiple/JJ
  writers/NNS
  ./.
  The/DT
  short/JJ
  film/NN
  stars/NNS
  Edward/NNP
  the/DT
  Spatula/NNP
  who/WP
  after/IN
  being/VBG
  fired/VBN
  from/IN
  his/PRP$
  job/NN
  ,/,
  joins/VBZ
  in/IN
  the/DT
  fight/NN
  against/IN
  the/DT
  evil/JJ
  spoons/NNS
  ./.
  This/DT
  premise/NN
  allows/VBZ
  for/IN
  some/DT
  funny/JJ
  content/NN
  near/IN
  the/DT
  beginning/NN
  ,/,
  but/CC
  is/VBZ
  barely/RB
  present/JJ
  for/IN
  the/DT
  remainder/NN
  of/IN
  the/DT
  feature/NN
  ./.
  This/DT
  film/NN
  's/POS
  15-minute/JJ
  running/NN
  time/NN
  is/VBZ
  absorbed/VBN
  by/IN
  some/DT
  odd-ball/JJ
  comedy/NN
  and/CC
  a/DT
  small/JJ
  musical/JJ
  number/NN
  ./.
  

[nltk_data]   Unzipping corpora/words.zip.


In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm.notebook import tqdm
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()
sia.polarity_scores(example)

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


{'neg': 0.078, 'neu': 0.767, 'pos': 0.155, 'compound': 0.8956}

In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm.notebook import tqdm
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()
sia.polarity_scores('I am so happy!')
# Run the polarity score on the entire dataset
res = {}
for i, row in tqdm(data.iterrows(), total=len(data)):
    text = row['review']
    res = sia.polarity_scores(text)

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


  0%|          | 0/50000 [00:00<?, ?it/s]

In [ ]:
# Run the polarity score on the entire dataset
res = []
for i, row in tqdm(data.iterrows(), total=len(data)):
    text = row['review']
    score = sia.polarity_scores(text)
    res.append(score)

# Convert the list of dictionaries to a DataFrame
vaders = pd.DataFrame(res)

# Combine sentiment scores with the original data
# Assuming 'data' is your DataFrame that includes a 'review' column
vaders = pd.concat([vaders, data.reset_index(drop=True)], axis=1)

vaders.head()


  0%|          | 0/50000 [00:00<?, ?it/s]

,neg,neu,pos,compound,review,sentiment
0,0.203,0.748,0.048,-0.9951,One of the other reviewers has mentioned that ...,1
1,0.053,0.776,0.172,0.9641,A wonderful little production. <br /><br />The...,1
2,0.094,0.714,0.192,0.9605,I thought this was a wonderful way to spend ti...,1
3,0.138,0.797,0.065,-0.9213,Basically there's a family where a little boy ...,0
4,0.052,0.801,0.147,0.9744,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [ ]:
#Tokenize text data pour que model undrestand our data :Xtrain & test
#tokenize convertit a word into integer tq ici il va prendre 50000 commons words in dataset

tokenizer = Tokenizer(num_words=5000)
#we fit this tokenize oncolonne qu on veut ENCODER !!
tokenizer.fit_on_texts(train_data['review'])
#puis on va coder jomal texts into sequence (liste) de meme long max:
X_train = pad_sequences(tokenizer.texts_to_sequences(train_data['review']),maxlen=200)
X_test = pad_sequences(tokenizer.texts_to_sequences(test_data['review']),maxlen=200)

In [ ]:
print(X_train)
print(X_test)

In [ ]:
Y_train = train_data['sentiment']
Y_test = test_data['sentiment']

In [ ]:
print(Y_train)
print(Y_test)

LSTM - Long Short-term Memory : maintain the sequence of input data cad reccurent :** it has the memory of others precedents words** so** it try to undrestand a sequence of words not chaque word seperately **

In [ ]:
#build the model
model = Sequential()
#this is the input layer
#5000 car 5000 words in tokenizer ecrite et 200 est taille de sequence mentionée en pad seq
#Construction du modèle LSTM : Couche d'Embedding : La couche d'embedding transforme les séquences de nombres entiers en vecteurs denses de taille fixe, ce qui permet au modèle de capturer les relations entre les mots.
model.add(Embedding(input_dim=5000,output_dim=128,input_length=200))
#second layer
#rappel unit === neurone
#dropout indice que model nest pas en overfitting :purpose of dropout is verifying that the model is not overfitting
model.add(LSTM(units=128,dropout=0.2,recurrent_dropout=0.2))
#outputlayer
model.add(Dense(units=1,activation='sigmoid'))





In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])


In [ ]:

model.fit(X_train,Y_train,batch_size=64,epochs=5,validation_split=0.2)

Model evaluation

In [ ]:
loss,accuracy = model.evaluate(X_test,Y_test)
print(f"loss : {loss}")
print(f"accuracy : {accuracy}")

Build a Predictive system

In [ ]:
def predict_sentiment(review):
  #firststep is tokenize and pad the review
  sequence = tokenizer.texts_to_sequences([review])
  padded_sequence = pad_sequences(sequence,maxlen=200)
  prediction = model.predict(padded_sequence)
  if prediction[0][0] > 0.5:
    return "positive"

  else:
    return "negative"

In [ ]:
new_review= "this movie was fantastic . I loved it ."
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is : {sentiment}")

In [ ]:
new_review= "this movie was not good ."
sentiment = predict_sentiment(new_review)
print(f"The sentiment of the review is : {sentiment}")